In [2]:
!pip install -q gradio scikit-learn pandas numpy


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

df = pd.read_csv("/content/credit_card_fraud_dataset.csv")

print("Shape:", df.shape)
print("Columns:", df.columns)

df.columns = df.columns.str.strip()

df["TransactionDate"] = pd.to_datetime(df["TransactionDate"], errors="coerce")
df["TransactionDate"] = df["TransactionDate"].astype("int64") // 10**9

df = pd.get_dummies(df, columns=["TransactionType", "Location"], drop_first=True)

df.drop("TransactionID", axis=1, inplace=True)

X = df.drop("IsFraud", axis=1)
y = df["IsFraud"]

with open("features.pkl", "wb") as f:
    pickle.dump(list(X.columns), f)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)


model = RandomForestClassifier(
    n_estimators=150,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

preds = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))
print(classification_report(y_test, preds))


with open("fraud_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Training completed successfully")


Shape: (100000, 7)
Columns: Index(['TransactionID', 'TransactionDate', 'Amount', 'MerchantID',
       'TransactionType', 'Location', 'IsFraud'],
      dtype='object')
Accuracy: 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     19800
           1       0.00      0.00      0.00       200

    accuracy                           0.99     20000
   macro avg       0.49      0.50      0.50     20000
weighted avg       0.98      0.99      0.99     20000

✅ Training completed successfully


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [4]:
import gradio as gr
import numpy as np
import pickle

model = pickle.load(open("fraud_model.pkl", "rb"))
scaler = pickle.load(open("scaler.pkl", "rb"))
features = pickle.load(open("features.pkl", "rb"))

def predict_fraud(*inputs):
    data = np.array(inputs).reshape(1, -1)
    data = scaler.transform(data)

    pred = model.predict(data)[0]
    prob = model.predict_proba(data)[0][1]

    if pred == 1:
        return f"🚨 Fraud Detected\nProbability: {prob:.2%}"
    else:
        return f"✅ Genuine Transaction\nFraud Probability: {prob:.2%}"

inputs = [gr.Number(label=col) for col in features]

app = gr.Interface(
    fn=predict_fraud,
    inputs=inputs,
    outputs=gr.Textbox(label="Prediction Result"),
    title="💳 Credit Card Fraud Detection System",
    description="Machine Learning based fraud detection (Google Colab)",
    theme="soft",
    css="""
    body {
        background: linear-gradient(120deg,#141e30,#243b55);
    }
    .gradio-container {
        border-radius: 20px;
        box-shadow: 0px 0px 25px rgba(0,0,0,0.4);
    }
    """
)

app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7f0eea01cab3252fa9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
